In [ ]:
from scipy.io import readsav
import cdflib
import pyspedas
from pyspedas import time_string, time_double, tinterpol
import pytplot
from pytplot import tplot, store_data, get_data ,tlimit,xlim,ylim,tplot_options,options,split_vec,cdf_to_tplot,divide,tplot_names,get_timespan, tplot_rename, time_datetime
#import wget
import warnings
from warnings import simplefilter 
simplefilter(action='ignore', category=DeprecationWarning)
warnings.filterwarnings("ignore")

import cdflib
from datetime import datetime,timedelta
import os.path
import bisect
import csv
from scipy import interpolate
import pandas as pd
import astropy.units as u,astropy.constants as const

#%matplotlib notebook
from mpl_toolkits import mplot3d
from mpl_toolkits.mplot3d import Axes3D
#import math functions 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors
import dateutil.parser
from matplotlib import ticker, cm

In [ ]:
#electron pads
spe_sf0_pad_cdf = '/Users/robertalexander/Dropbox/__Collaborations/_NASA/__MH_Investigation/Downloaded_PSP_Data/High_Resolution_Electrons/psp_swp_spe_af0_L3_pad_20230928_v04.cdf'
spe_sf0_pad_tplot = cdf_to_tplot(spe_sf0_pad_cdf,get_support_data=True)


#Access electron strahl pitch-angle distribution (PAD)
epad_data = get_data('psp_spe_EFLUX_VS_PA_E')
epad_PA = get_data('psp_spe_PITCHANGLE')

if epad_data is None or epad_PA is None:
    print("Error: Failed to load electron PAD data")
    # Handle the error appropriately (e.g., exit the script or load default data)
else:
    epad_times = epad_data.times
    epad_vals = epad_data.y
    epad_PA_vals = epad_PA.y

    #Note for E1-E9, electron strahl energy bin is 8, for E10 and above, it's 12
    energy_index = 12
    epad_strahl = epad_vals[:,:,energy_index]

    datetime_spe = time_datetime(epad_times)
    times_spe_repeat = np.repeat(np.expand_dims(datetime_spe,1),12,1)

    #compute centroids
    centroids_epad = np.ma.average(epad_PA_vals, 
                           weights=epad_strahl, 
                           axis=1)

In [ ]:
#Proton temperature anisotropy from Temperature Tensor

#Access tensor elements
T_Tens = get_data('psp_spi_T_TENSOR_INST')
T_XX = T_Tens.y[:,0]
T_YY = T_Tens.y[:,1]
T_ZZ = T_Tens.y[:,2]
T_XY = T_Tens.y[:,3]
T_XZ = T_Tens.y[:,4]
T_YZ = T_Tens.y[:,5]

T_YX = T_XY
T_ZX = T_XZ
T_ZY = T_YZ

#Access magnetic field in span-I coordinates
B_spi = get_data('psp_MAGF_INST')
B_X = B_spi.y[:,0]
B_Y = B_spi.y[:,1]
B_Z = B_spi.y[:,2]
B_mag_XYZ = np.sqrt(B_X**2 + B_Y**2 + B_Z**2)

#Project Tensor onto B field, find perpendicular and parallel components
T_parallel=[]
T_perpendicular=[]
Anisotropy=[]
for i in range(len(B_X)):
    Sum_1=B_X[i]*B_X[i]*T_XX[i]
    Sum_2=B_X[i]*B_Y[i]*T_XY[i]
    Sum_3=B_X[i]*B_Z[i]*T_XZ[i]
    Sum_4=B_Y[i]*B_X[i]*T_YX[i]
    Sum_5=B_Y[i]*B_Y[i]*T_YY[i]
    Sum_6=B_Y[i]*B_Z[i]*T_YZ[i]
    Sum_7=B_Z[i]*B_X[i]*T_ZX[i]
    Sum_8=B_Z[i]*B_Y[i]*T_ZY[i]
    Sum_9=B_Z[i]*B_Z[i]*T_ZZ[i]    
    T_para=((Sum_1+Sum_2+Sum_3+Sum_4+Sum_5+Sum_6+Sum_7+Sum_8+Sum_9)/(B_mag_XYZ[i])**2)
    Trace_Temp=(T_XX[i]+T_YY[i]+T_ZZ[i])
    T_perp=(Trace_Temp-T_para)/2.0
    T_parallel.append((Sum_1+Sum_2+Sum_3+Sum_4+Sum_5+Sum_6+Sum_7+Sum_8+Sum_9)/(B_mag_XYZ[i])**2)
    T_perpendicular.append(T_perp)
    Anisotropy.append(T_perp/T_para)

In [ ]:

ax_epad = 0
ax_tpar = 1
ax_tperp = 1
ax_trat = 2
plt.rcParams.update({'font.size': 32})

fig, ax = plt.subplots(8, 1, sharex=True, figsize=(30, 25),layout='compressed')
s=5
alpha=.7
linewidth = 2.0



s_epad=ax[ax_epad].pcolormesh(times_spe_repeat,epad_PA_vals,
         np.log10(epad_strahl),cmap='gist_stern',vmin=8,vmax=11)
cbar_epad = plt.colorbar(s_epad,label=f'Flux ($eV/(cm^2 \\ s \\ sr \\ eV)$')
ax[ax_epad].plot(datetime_spe,centroids_epad,color='black', linestyle='-', marker='o', markersize=4)
ax[ax_tpar].plot(time_datetime(T_Tens.times),T_parallel,color='deepskyblue',label = '$T_\parallel$')
ax[ax_tperp].plot(time_datetime(T_Tens.times),T_perpendicular,color='hotpink',label = '$T_\perp$')
ax[ax_trat].plot(time_datetime(T_Tens.times),Anisotropy,color='mediumspringgreen',label = '$T_\perp/T_\parallel$')



ax[ax_epad].set(
    ylabel = f'Electron \n Strahl \n PAD ($^/circ$)'
    #ylim = [-500,500]
)


ax[ax_trat].set(
    ylabel=r'$T_\perp/T_\parallel$',
    ylim = [0,6]
)

ax[ax_tpar].legend()
ax[ax_tperp].legend()

plt.show()